In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


In [6]:
df = sns.load_dataset('iris')

In [3]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [8]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [3]:
df.head()
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [4]:
np.unique(df['species'])
df['species'] = df['species'].replace({'setosa':0, 'versicolor':1, 'virginica':2})
from sklearn.model_selection import train_test_split
X = df.iloc[:,:-1]
y = df['species']
trainX,testX,trainy,testy = train_test_split(X,y,test_size=0.3,random_state=28)
trainX.shape,trainy.shape

((105, 4), (105,))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [5]:
model = LogisticRegression(max_iter=1000)
model.fit(trainX,trainy)
model.coef_
model.intercept_
training_pred = model.predict(trainX)

In [6]:
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(trainy,training_pred)
confusion_matrix(trainy,training_pred)
print(classification_report(trainy,training_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        38
           1       0.97      0.97      0.97        31
           2       0.97      0.97      0.97        36

    accuracy                           0.98       105
   macro avg       0.98      0.98      0.98       105
weighted avg       0.98      0.98      0.98       105



In [7]:
def class_tptnfpfn(a,p):
    result = pd.crosstab(p,a , rownames=['pred'], colnames=['actual'])
    tmp = []
    for i in range(len(np.unique(a))):
        tp = result.values[i,i]
        tn = result.drop(index=i,axis=0).drop(labels=i,axis=1).values.sum().sum()
        fp = result.iloc[i,:].drop(i).values.sum().sum()
        fn = result.iloc[:,i].drop(i).values.sum().sum()
        tmp.append([f'{i}',tp,tn,fp,fn])
    cm_final = pd.DataFrame(tmp)
    cm_final.columns = ['Classes',"TP","TN","FP","FN"]
    cm_final['accuray'] = (cm_final['TP']+cm_final['TN'])/(cm_final['TP']+cm_final['TN']+cm_final['FP']+cm_final['FN'])
    cm_final['precision'] = (cm_final['TP'])/(cm_final['TP']+cm_final['FP'])
    cm_final['recall'] = (cm_final['TP'])/(cm_final['TP']+cm_final['FN'])
    cm_final['f1-score'] = (2*cm_final['precision']*cm_final['recall'])/(cm_final['precision']+cm_final['recall'])
    cm_final = pd.concat([cm_final,pd.DataFrame(cm_final.iloc[:,-4:].mean()).T],axis=0)
    cm_final = cm_final.fillna(" ")
    return cm_final

In [16]:
class_tptnfpfn(a=trainy,p=training_pred)
testing_pred = model.predict(testX)
class_tptnfpfn(a=testy,p=testing_pred)

,Classes,TP,TN,FP,FN,accuray,precision,recall,f1-score
0,0,12.0,33.0,0.0,0.0,1.000000,1.000000,1.000000,1.000000
1,1,17.0,26.0,0.0,2.0,0.955556,1.000000,0.894737,0.944444
2,2,14.0,29.0,2.0,0.0,0.955556,0.875000,1.000000,0.933333
0,,,,,,0.970370,0.958333,0.964912,0.959259
